# Classifications

In [ ]:
import math
import time
import datetime
import os
import csv
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow_probability import edward2 as ed
from vqbnn.vqbnn import VQBNN
from vqbnn.och import OCH

In [ ]:
savefig = False

plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

if not savefig:
    plt.rcParams["figure.figsize"] = (4, 4)
    plt.rcParams["font.size"] = 15
    plt.rcParams["figure.titlesize"] = 25
    plt.rcParams["axes.labelsize"] = 20
    plt.rcParams["xtick.labelsize"] = 15
    plt.rcParams["ytick.labelsize"] = 15
    plt.rcParams["legend.fontsize"] = 13
    plt.rcParams["lines.linewidth"] = 2
else:
    plt.rcParams["figure.figsize"] = (8, 8)
    plt.rcParams["font.size"] = 30
    plt.rcParams["axes.labelsize"] = 53
    plt.rcParams["xtick.labelsize"] = 40
    plt.rcParams["ytick.labelsize"] = 40
    plt.rcParams["legend.fontsize"] = 28
    plt.rcParams["lines.linewidth"] = 4

Constants and hyperparameters:

In [ ]:
# paths
CLASSIFICATION_FLAG = "classifications"
DATASET_PATH = 'datasets/'
BNN_PATH = 'models_checkpoints/bnn'
DAT_PATH = "leaderboard/%s/" % CLASSIFICATION_FLAG

# experiments
seed = 30
tf.random.set_seed(seed)
np.random.seed(seed)
POSTERIOR_NO = 30
CONFIDENCE_CUTOFF = 0.9

# och parameters
och_x1_params = {'k': 5, 'l': 5.0, 's': 1.0}
och_x_params = {'k': 10, 'l': 0.01, 's': 1.0}
och_y_params = {'k': 10, 'l': 0.01, 's': 1.0}

# style
alpha = 0.12
colors = ["tab:blue", "tab:green", "tab:purple", "tab:red"]
labels = ["DNN", "MU", "DU", "VQ-BNN"]
guide_linestyle=(0, (1, 1))
linestyles = [(0, (5, 1)), 'solid', (0, (5, 1)), 'solid']

In [ ]:
def bnn_categorical(model, xs):
    logits = model(xs)
    return tfd.Categorical(logits=logits)

In [ ]:
def train_epoch(model, x_train, y_train, batch, optimizer, loss_ftn, *loss_metrics):
    indexes_batch = np.array_split(np.random.permutation(len(x_train)), len(x_train) / batch + 1)
    indexes_batch = [indexes for indexes in indexes_batch if len(indexes) > 0]

    for indexes in indexes_batch:
        xs = tf.stack([x_train[index] for index in indexes])
        ys = tf.stack([y_train[index] for index in indexes])
        train_step(model, xs, ys, optimizer, loss_ftn, *loss_metrics)
    
def train_step(model, x_batch, y_batch, optimizer, loss_ftn, *loss_metrics):
    with tf.GradientTape() as tape:
        losses = loss_ftn(model, x_batch, y_batch)
    gradients = tape.gradient(losses[0], model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    for loss, loss_metric in zip(losses, loss_metrics):
        loss_metric(loss)
        
def bnn_categorical_loss_ftn(model, xs, ys, length):
    ys = tf.squeeze(ys, axis=-1)
    nll = - tf.reduce_mean(bnn_categorical(model, xs).log_prob(ys))
    kl = sum(model.losses) / length
    loss = nll + kl
    return loss, nll

In [ ]:
def evaluate_nn(model, x_test, y_test, depth, batch=1, stat_period=1, sample_no=POSTERIOR_NO, cutoff=CONFIDENCE_CUTOFF, print_stat=False):
    cce_object = tf.keras.losses.CategoricalCrossentropy()
    mse_metric = tf.keras.metrics.Mean(name='mse')
    mse_90_metric = tf.keras.metrics.Mean(name='mse-90')
    cce_metric = tf.keras.metrics.Mean(name='cce')
    cov_metric = tf.keras.metrics.Mean(name='cov')

    indexes_batch = np.array_split(range(len(x_test)), len(x_test) / batch + 1)
    indexes_batch = [indexes for indexes in indexes_batch if len(indexes) > 0]
    for i, indexes in enumerate(indexes_batch):
        start_time = time.time()
        xs = tf.stack([x_test[index] for index in indexes])
        ys = tf.stack([y_test[index] for index in indexes])
        ys = tf.one_hot(tf.reshape(ys, [-1]), depth)
        
        prediction = [model(xs) for _ in range(sample_no)]
        prediction = tf.nn.softmax(prediction)
        prediction = tf.math.reduce_mean(prediction, axis=0)
        filtered = [(p, y) for p, y in zip(prediction, ys) if tf.math.reduce_max(p) >= cutoff]
        prediction_cutoff, ys_cutoff = tf.stack([p for p, _ in filtered]), tf.stack([y for _, y in filtered])
        
        if print_stat:
            print("ys:", np.array([y_test[index][0] for index in indexes]))
            print("result: ", tf.math.argmax(prediction, axis=-1))
            print("confidence: ", tf.math.reduce_max(prediction, axis=-1))
        
        mse = tf.keras.losses.MSE(ys, prediction)
        cce = cce_object(ys, prediction)
        mse_cutoff = tf.keras.losses.MSE(ys_cutoff, prediction_cutoff)

        mse_metric(mse)
        cce_metric(cce)
        cov_metric(len(prediction_cutoff) / len(prediction))
        if len(filtered) > 0:
            mse_90_metric(mse_cutoff)
    
        if ((i + 1) % stat_period is 0) or ((i + 1) is len(indexes_batch)):
            template = "(%s, %.1f sec/epoch), Epoch %d/%d, MSE: %.4f, MSE-90: %.4f, CCE: %.4f, Cov-90: %.4f %%"
            print(template % (datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                              (time.time() - start_time) / stat_period,
                              i + 1,
                              len(indexes_batch),
                              mse_metric.result(),
                              mse_90_metric.result(),
                              cce_metric.result(),
                              cov_metric.result() * 100))

In [ ]:
def evaluate_vqbnn(vqbnn, x_test, y_test, depth, stat_period=1, sample_no=POSTERIOR_NO, cutoff=CONFIDENCE_CUTOFF):
    cce_object = tf.keras.losses.CategoricalCrossentropy()
    mse_metric = tf.keras.metrics.Mean(name='mse')
    mse_90_metric = tf.keras.metrics.Mean(name='mse-90')
    cce_metric = tf.keras.metrics.Mean(name='cce')
    cov_metric = tf.keras.metrics.Mean(name='cov')

    for i, (x, y) in enumerate(zip(x_test, y_test)):
        start_time = time.time()
        ys = tf.one_hot(y, depth)
        vqbnn.update(x)
        
        if sum([w for _, w in vqbnn.och_y.cws()]) > 0:
            prediction = [tf.nn.softmax(c) * w for c, w in vqbnn.och_y.cws()]
            prediction = tf.math.add_n(prediction)
            filtered = [(p, y) for p, y in zip(prediction, ys) if tf.math.reduce_max(p) >= cutoff]
            prediction_cutoff, ys_cutoff = tf.stack([p for p, _ in filtered]), tf.stack([y for _, y in filtered])

            mse = tf.keras.losses.MSE(ys, prediction)
            cce = cce_object(ys, prediction)
            mse_cutoff = tf.keras.losses.MSE(ys_cutoff, prediction_cutoff)
            
            mse_metric(mse)
            cov_metric(len(prediction_cutoff) / len(prediction))
            cce_metric(cce)
            if len(filtered) > 0:
                mse_90_metric(mse_cutoff)
    
        if ((i + 1) % stat_period is 0) or ((i + 1) is len(x_test)):
            template = "(%s, %.1f sec/epoch), Epoch %d/%d, MSE: %.4f, MSE-90: %.4f, CCE: %.4f, Cov-90: %.4f"
            print(template % (datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                              (time.time() - start_time) / stat_period,
                              i + 1,
                              len(x_test),
                              mse_metric.result(),
                              mse_90_metric.result(),
                              cce_metric.result(),
                              cov_metric.result() * 100))

In [ ]:
LAYER_NAMES = DENSE_1, DENSE_2, DENSE_3 = "dense_1", "dense_2", "dense_3"

def create_dnn(classes_no, unit=50):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(unit, activation=tf.nn.relu, name=DENSE_1),
        tf.keras.layers.Dense(unit, activation=tf.nn.relu, name=DENSE_2),
        tf.keras.layers.Dense(unit, activation=tf.nn.relu, name=DENSE_3),
        tf.keras.layers.Dense(classes_no, name=DENSE_3)
    ])

def create_bnn(classes_no, unit=50):
    return tf.keras.Sequential([
        tfp.layers.DenseFlipout(unit, activation=tf.nn.relu, name=DENSE_1),
        tfp.layers.DenseFlipout(unit, activation=tf.nn.relu, name=DENSE_2),
        tfp.layers.DenseFlipout(classes_no, name=DENSE_3)
    ])

def freeze(bnn, input_dim, posterior_no):
    kernel_d1, bias_d1 = [], []
    kernel_d2, bias_d2 = [], []
    kernel_d3, bias_d3 = [], []
    
    layer1 = bnn.get_layer(DENSE_1) 
    layer2 = bnn.get_layer(DENSE_2) 
    layer3 = bnn.get_layer(DENSE_3) 

    layer1.kernel_posterior_tensor_fn = lambda d: append(kernel_d1, d.sample())
    layer1.bias_posterior_tensor_fn = lambda d: append(bias_d1, d.sample())
    layer2.kernel_posterior_tensor_fn = lambda d: append(kernel_d2, d.sample())
    layer2.bias_posterior_tensor_fn = lambda d: append(bias_d2, d.sample())
    layer3.kernel_posterior_tensor_fn = lambda d: append(kernel_d3, d.sample())
    layer3.bias_posterior_tensor_fn = lambda d: append(bias_d3, d.sample())
        
    _ = [bnn(tf.random.normal((1, input_dim))) for _ in range(posterior_no)]
    
    layer1.kernel_posterior_tensor_fn = lambda d: kernel_d1[random.randint(0, posterior_no - 1)]
    layer1.bias_posterior_tensor_fn = lambda d: bias_d1[random.randint(0, posterior_no - 1)]
    layer2.kernel_posterior_tensor_fn = lambda d: kernel_d2[random.randint(0, posterior_no - 1)]
    layer2.bias_posterior_tensor_fn = lambda d: bias_d2[random.randint(0, posterior_no - 1)]
    layer3.kernel_posterior_tensor_fn = lambda d: kernel_d3[random.randint(0, posterior_no - 1)]
    layer3.bias_posterior_tensor_fn = lambda d: bias_d3[random.randint(0, posterior_no - 1)]
    
def append(xs, x):
    xs.append(x)
    return x

Table of Contents
* [Occupancy Detection Data Set (Classification)](#Occupancy-Detection-Data-Set-(Classification))
* [EMG Data for Gestures Data Set (Classification)](#EMG-Data-for-Gestures-Data-Set-(Classification))
* [Localization Data for Person Activity Data Set (Classification)](#Localization-Data-for-Person-Activity-Data-Set-(Classification))

## [Occupancy Detection Data Set](https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+) (Classification)

In [ ]:
OCC_FLAG = "occupancy"
OCC_INPUT_DIM = 5
OCC_CLASS_NO = 2

In [ ]:
train_file_names, test_file_names = ["datatraining.txt"], ["datatest.txt", "datatest2.txt"]

def occupancy_data(file_name):
    occupancy_path = DATASET_PATH + "%s/" % OCC_FLAG
    x, y = [], []
    with open(occupancy_path + file_name, 'r') as csvfile:
        file_reader = csv.reader(csvfile, delimiter=',')
        header = next(file_reader)
        for row in file_reader:
            x.append(tf.constant([float(v) for v in row[2: 7]]))
            y.append(tf.constant([int(row[7])]))
    return x, y

In [ ]:
x_train, y_train = occupancy_data(train_file_names[0])
x_test, y_test = tuple(zip(*[occupancy_data(test_file_name) for test_file_name in test_file_names]))
x_test, y_test = tf.concat(x_test, 0), tf.concat(y_test, 0)

print("Train dataset: %d, Testset size: %d" % (len(x_train), len(x_test)))
print("x sample: ", x_train[0].numpy())
print("y sample: ", y_train[0].numpy())

### Train Models

In [ ]:
bnn = create_bnn(OCC_CLASS_NO)

In [ ]:
epochs, batch = 50, 100

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_ftn = lambda model, xs, ys: bnn_categorical_loss_ftn(model, xs, ys, len(x_train))
train_loss = tf.keras.metrics.Mean(name='train_loss')
nll_loss = tf.keras.metrics.Mean(name='nll_loss')

for epoch in range(epochs):
    time1 = time.time()
    train_epoch(bnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss, nll_loss)
    time2 = time.time()
        
    if (epoch + 1) % 1 == 0:
        template = '(%.2f sec) Epoch: %d, Loss: %.4f, NLL: %.4f'
        print(template % (time2 - time1,
                          epoch + 1,
                          train_loss.result(),
                          nll_loss.result()))
        train_loss.reset_states()
        nll_loss.reset_states()
        
bnn.save_weights("%s-%s" % (BNN_PATH, OCC_FLAG), save_format='tf')

### Test Models

Test MU:

In [ ]:
bnn = create_bnn(OCC_CLASS_NO)
bnn.load_weights("%s-%s" % (BNN_PATH, OCC_FLAG))

In [ ]:
evaluate_nn(bnn, x_test, y_test, OCC_CLASS_NO, batch=1, stat_period=1)

Test VQ-BNN:

In [ ]:
och_x_params = {'k': 10, 'l': 1.3, 's': 1.0}
och_y_params = {'k': 10, 'l': 1.3, 's': 1.0}

vqbnn_op = create_bnn(OCC_CLASS_NO)
vqbnn_op.load_weights("%s-%s" % (BNN_PATH, OCC_FLAG))
freeze(vqbnn_op, OCC_INPUT_DIM, POSTERIOR_NO)

x_dims, y_dims = [OCC_INPUT_DIM], [OCC_CLASS_NO]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
vqbnn = VQBNN(lambda x: vqbnn_op(tf.stack([x[0]])), och_x_1, och_x, och_y)

evaluate_vqbnn(vqbnn, x_test, y_test, OCC_CLASS_NO, stat_period=10)

## [EMG Data for Gestures Data Set](https://archive.ics.uci.edu/ml/datasets/EMG+data+for+gestures) (Classification)

In [ ]:
EMG_FLAG = "emg"
EMG_INPUT_DIM = 8
EMG_CLASS_NO = 8

In [ ]:
def emg_data(dataset_no):
    emg_path = DATASET_PATH + "%s/" % EMG_FLAG
    channels, labels = [], []
    dat_path = emg_path + "%02d/" % dataset_no
    for file_name in os.listdir(dat_path):
        with open(dat_path + file_name, 'r') as csvfile:
            file_reader = csv.reader(csvfile, delimiter='\t')
            header = next(file_reader)
            for row in file_reader:
                if len(row) == 10:
                    channels.append(tf.constant([float(v) for v in row[1:9]]))
                    labels.append(tf.constant([int(row[9])]))
    return channels, labels

In [ ]:
regularizer = 10e4

# x_train, y_train = tuple(zip(*[emg_data(i) for i in range(1, 33)]))
x_train, y_train = tuple(zip(*[emg_data(i) for i in range(1, 10)]))
x_train, y_train = tf.concat(x_train, 0), tf.concat(y_train, 0)
x_train = x_train * regularizer
# x_test, y_test = tuple(zip(*[emg_data(i) for i in range(33, 37)]))
x_test, y_test = tuple(zip(*[emg_data(i) for i in range(36, 37)]))
x_test, y_test = tf.concat(x_test, 0), tf.concat(y_test, 0)
x_test = x_test * regularizer

print("train dataset: %d" % len(x_train))
print("x sample: ", x_train[0].numpy())
print("y sample: ", y_train[0].numpy())
print("test dataset: %d" % len(x_test))
print("x sample: ", x_test[0].numpy())
print("y sample: ", y_test[0].numpy())

### Train Models

Train BNN:

In [ ]:
bnn = create_bnn(EMG_CLASS_NO)

In [ ]:
epochs, batch = 30, 300

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_ftn = lambda model, xs, ys: bnn_categorical_loss_ftn(model, xs, ys, len(x_train))
train_loss = tf.keras.metrics.Mean(name='train_loss')
nll_loss = tf.keras.metrics.Mean(name='nll_loss')

for epoch in range(epochs):
    time1 = time.time()
    train_epoch(bnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss, nll_loss)
    time2 = time.time()
    
    if (epoch + 1) % 1 == 0:
        template = '({} sec) Epoch {}, Loss: {}, NLL: {}'
        print(template.format(time2 - time1,
                              epoch + 1,
                              train_loss.result(),
                              nll_loss.result()))
        train_loss.reset_states()
        nll_loss.reset_states()

    if (epoch + 1) % 10 == 0:
        bnn.save_weights("%s-%s" % (BNN_PATH, EMG_FLAG), save_format='tf')
        print("Snapshot saved.")
        
bnn.save_weights("%s-%s" % (BNN_PATH, EMG_FLAG), save_format='tf')

### Test Models

Test MU:

In [ ]:
bnn = create_bnn(EMG_CLASS_NO)
bnn.load_weights("%s-%s" % (BNN_PATH, EMG_FLAG))

In [ ]:
evaluate_nn(bnn, x_test, y_test, EMG_CLASS_NO, batch=100, stat_period=1, cutoff=0.9, print_stat=False)

Test VQ-BNN:

In [ ]:
och_x_params = {'k': 10, 'l': 1.3, 's': 1.0}
och_y_params = {'k': 10, 'l': 1.3, 's': 1.0}

vqbnn_op = create_bnn(EMG_CLASS_NO)
vqbnn_op.load_weights("%s-%s" % (BNN_PATH, EMG_FLAG))
freeze(vqbnn_op, EMG_INPUT_DIM, POSTERIOR_NO)

x_dims, y_dims = [EMG_INPUT_DIM], [EMG_CLASS_NO]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
vqbnn = VQBNN(lambda x: vqbnn_op(tf.stack([x[0]])), och_x_1, och_x, och_y)

evaluate_vqbnn(vqbnn, x_test, y_test, EMG_CLASS_NO, stat_period=300)

## [Localization Data for Person Activity Data Set](https://archive.ics.uci.edu/ml/datasets/Localization+Data+for+Person+Activity) (Classification)

In [ ]:
LOC_FLAG = "localization"
LOC_INPUT_DIM = 4
LOC_CLASS_NO = 11
TAG = ["010-000-024-033", "010-000-030-096", "020-000-033-111", "020-000-032-221"]
ACTIVITY = ["walking","falling","lying down","lying","sitting down","sitting","standing up from lying","on all fours","sitting on the ground","standing up from sitting","standing up from sitting on the ground"]

In [ ]:
def localization_data():
    file_path = DATASET_PATH + "%s/%s" % (LOC_FLAG, "ConfLongDemo_JSI.txt")
    xs, ys = [], []
    with open(file_path, 'r') as csvfile:
        file_reader = csv.reader(csvfile, delimiter=',')
        header = next(file_reader)
        for row in file_reader:
            xs.append(tf.constant([TAG.index(row[1])] + [float(v) for v in row[4:7]]))
            ys.append(tf.constant([ACTIVITY.index(row[7])]))
    return xs, ys

In [ ]:
xs, ys = localization_data()
x_len, train_len = len(xs), int(len(xs) * 0.9)
x_train, y_train, x_test, y_test = xs[:train_len], ys[:train_len], xs[train_len:], ys[train_len:]

print("train dataset: %d, test datasets: %d" % (len(x_train), len(x_test)))
print("x sample: ", x_train[0].numpy())
print("y sample: ", y_train[0].numpy())

### Train Models

Train BNN:

In [ ]:
epochs, batch = 30, 3

bnn = create_bnn(LOC_CLASS_NO)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_ftn = lambda model, xs, ys: bnn_categorical_loss_ftn(model, xs, ys, len(x_train))
train_loss = tf.keras.metrics.Mean(name='train_loss')
nll_loss = tf.keras.metrics.Mean(name='nll_loss')

for epoch in range(epochs):
    time1 = time.time()
    train_epoch(bnn, x_train, y_train, batch, optimizer, loss_ftn, train_loss, nll_loss)
    time2 = time.time()
        
    if (epoch + 1) % 1 == 0:
        template = 'Epoch {}, Loss: {}, NLL: {}'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              nll_loss.result()))
        train_loss.reset_states()
        nll_loss.reset_states()
        
bnn.save_weights("%s-%s" % (BNN_PATH, LOC_FLAG), save_format='tf')

### Test

Test BNN:

In [ ]:
bnn = create_bnn(LOC_CLASS_NO)
bnn.load_weights("%s-%s" % (BNN_PATH, LOC_FLAG))

In [ ]:
evaluate_nn(bnn, x_test, y_test, LOC_CLASS_NO, batch=100, stat_period=1)

Test VQ-BNN:

In [ ]:
och_x_params = {'k': 10, 'l': 1.3, 's': 1.0}
och_y_params = {'k': 10, 'l': 1.3, 's': 1.0}

vqbnn_op = create_bnn(LOC_CLASS_NO)
vqbnn_op.load_weights("%s-%s" % (BNN_PATH, LOC_FLAG))
freeze(vqbnn_op, LOC_INPUT_DIM, POSTERIOR_NO)

x_dims, y_dims = [LOC_INPUT_DIM], [LOC_CLASS_NO]
och_x = OCH(**och_x_params, dims=x_dims, hash_no=1)
och_y = OCH(**och_y_params, dims=y_dims, hash_no=1)
och_x_1 = OCH(**och_x1_params, dims=x_dims[1:], hash_no=3, cs=[])
vqbnn = VQBNN(lambda x: vqbnn_op(tf.stack([x[0]])), och_x_1, och_x, och_y)

evaluate_vqbnn(vqbnn, x_test, y_test, LOC_CLASS_NO, stat_period=100)